In [1]:
import random, gym
import numpy as np
from deap import algorithms, base, creator, tools

## Simülasyon ortamı

In [2]:
env = gym.make("CartPole-v1")
observation_space_dim = env.observation_space.shape[0]
action_space_dim = env.action_space.n

#### Tekrarlanabilirlik için seed ayarlanır

In [3]:
SEED_VALUE = 64
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
env.seed(SEED_VALUE)

[64]

#### Simülasyonu çalıştırmak için ortak fonksiyon

In [4]:
def run_env(act_fn, n_episode=1, render=False):
	# Episode döngüsü
	totalRewards = 0
	for episode in range(n_episode):
		state = env.reset()
		done = False

		# Timestep döngüsü
		while not done:
			if render: env.render()

			# act_fn adlı fonksiyon parametreyi çağırarak aksiyonu al
			action = act_fn(state)
			next_state, reward, done, info = env.step(action)

			totalRewards += reward
			state = next_state
	return totalRewards

## Yapay Sinir Ağı

#### Ara katman ünite sayısı

In [5]:
nn_hidden_unit = 8
w1_ndim = (observation_space_dim*nn_hidden_unit)
w2_ndim = (nn_hidden_unit*action_space_dim)

#### Verilen bireyin genotipini yapay sinir ağı parametreleri olarak kullanarak, state vektörünü ağa feed-forward eder

In [6]:
def nn_forward(individual, state):
	# State'in 2 seviyeli array olduğundan emin ol: [?, observation_space_dim]
	if len(state.shape) == 1: state = np.array([state])
	assert len(state.shape) == 2

	# Yapay sinir ağı parametrelerini genotip olan 1D vektörden çıkar
	individual = np.array(individual)
	w1 = individual[:w1_ndim]
	b1 = individual[w1_ndim]
	w2 = individual[w1_ndim+1:-1]
	b2 = individual[-1]

	# Ağırlıkları matris çarpımı için yeniden şekillendir
	w1 = np.reshape(w1, (observation_space_dim, nn_hidden_unit))
	w2 = np.reshape(w2, (nn_hidden_unit, action_space_dim))

	# Sigmoid fonksiyonu
	sigmoid = lambda x: (1 / (1 + np.exp(-x)))

	# Feed-forward
	h1 = sigmoid(np.dot(state, w1) + b1)
	output = sigmoid(np.dot(h1, w2) + b2)
	return np.argmax(output)

## Genetik Algoritmalar

### Hiperparametreler

In [7]:
# Her bireyin gen uzunluğu (ağın parametre sayısı)
n_features = (w1_ndim + w2_ndim + 2)

# Simüle edilecek nesil sayısı
n_generation = 25

# Popülasyon boyutu (birey sayısı)
n_population = 32

# Seleksiyon turnuvasındaki birey sayısı
selectionTournamentSize = 3

# Çaprazlama ve mutasyon olasılıkları
crossingoverProbability = 0.50
mutationProbability = 0.10

In [8]:
# Uygunluk ve Birey sınıflarını oluştur
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

# Bireylerin genotiplerini ve popülasyonun basitçe birey listesi olduğunu tanımla
toolbox = base.Toolbox()
toolbox.register("attr_weights_n_biases", random.gauss, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_weights_n_biases, n_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#### Uygunluk fonksiyonunu tanımla

In [9]:
def calculateFitness(individual):
	individualEpisodeRewards = run_env(
		# Aksiyon fonksiyonumuz, bireyin genotipini kullanarak yapay sinir ağını çalıştırır
		act_fn=(lambda state: nn_forward(individual, state)),
		# Her uygunluk hesabında ortamı 3 episode çalıştır
		n_episode=3
	)
	# tuple tipinde çevirmeli
	return (individualEpisodeRewards, )

In [10]:
# Uygunluk fonksiyonunu kaydet
toolbox.register("evaluate", calculateFitness)

# Hangi çaprazlama, mutasyon ve seçilim yöntemlerinin kullanılacağını tanımla
toolbox.register("mate", tools.cxUniform, indpb=0.50)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=selectionTournamentSize)

#### Popülasyonu oluştur

In [11]:
population = toolbox.population(n=n_population)
stats = tools.Statistics(lambda ind: ind.fitness.values[0])
hallOfFame = tools.HallOfFame(1)

#### GA simülasyonu öncesi rastgele bir bireyle ortamı çalıştır

In [12]:
run_env(
	act_fn=(lambda state: nn_forward(
		random.choice(population),
		state
	)),
	n_episode=10,
	render=True
)

153.0

## Simülasyon başlasın!

In [13]:
finalPopulation, logs = algorithms.eaSimple(
	population=population,
	toolbox=toolbox,
	halloffame=hallOfFame,
	stats=stats,
	ngen=n_generation,
	cxpb=crossingoverProbability,
	mutpb=mutationProbability,
	verbose=True
)

gen	nevals
0  	32    
1  	15    
2  	17    
3  	14    
4  	15    
5  	17    
6  	22    
7  	20    
8  	19    
9  	13    
10 	22    
11 	15    
12 	19    
13 	17    
14 	21    
15 	16    
16 	19    
17 	19    
18 	16    
19 	16    
20 	12    
21 	18    
22 	14    
23 	17    
24 	12    
25 	20    


### En iyi bireyle simülasyonu çalıştır

In [14]:
best = hallOfFame[0]
print("[+] En iyi uygunluk değeri: {}".format(best.fitness.values[0]))
print("Genotip: ", best)

run_env(
	act_fn=(lambda state: nn_forward(best, state)),
	n_episode=5,
	render=True
)
env.close()

[+] En iyi uygunluk değeri: 1500.0
Genotip:  [0.09109158780977948, -0.29858014725238036, 0.6294105328746935, 0.1568631379018537, -0.1883482020339822, 2.2327974225272604, 1.3995720972839263, -0.35413810557596687, 2.4397432302727893, -1.0072705401817257, 0.34210371400769113, 1.0625560697954215, -0.494609903815069, 1.261881277227563, 0.9149917505737201, -0.530551516345594, -0.7103199483218117, 0.636496255526635, 1.493047161102028, -0.5633082001651568, 0.2945375654918546, 0.6842303871897123, -0.6382357447422995, -0.49009917221582416, -1.8098453746951857, 2.135655485733688, -0.0514269505998888, -0.5857764586617262, -0.3675898384451499, 0.4683190083160991, -0.846697382197066, 0.336550613060773, -1.1106473613436048, -0.5275102000580876, -0.2960636571308189, 0.0943059166064184, 0.738881451455288, -0.3359748190757696, 0.4248690321812363, -0.22611891206490897, -0.288596426653344, 0.5472552521504789, 0.3302625278313965, -0.03737743806714264, 0.7967697710010319, 0.6047836341446641, -0.434761185486